In [107]:
from prompt import Prompt, RobertaPrompt

### Define the Prompt

We define a prompt that can perform two tasks given a topic and an argument <br>
1) classify whether the argument is a fallacy, and if so what fallacy the argument contains <br>
2) decide whether the argument supports or refutes the topic <br>

In [189]:
templates = {"fallacy": "fallacy task. Topic: {} Text: {} This contains the fallacy: {}", 
             "stance": "procon task. Topic: {} Text: {} Has the relation: {}"}

def fallacy_policy(pred):
  fallacies = {'AppealtoEmotion', 'RedHerring', 'NoFallacy', 'IrrelevantAuthority','AdHominem','HastyGeneralization'}
  if pred not in fallacies: return 'UNKNOWN'
  return pred
def stance_policy(pred):
  if pred not in {"support", "contradict"}: return "UNKNOWN"
  return pred

policies = {"fallacy": fallacy_policy, "stance": stance_policy}

argument_prompt = Prompt(templates, policies)

RobertaPrompt uses this class to convert a sample into it's desire prompt - for example

In [123]:
argument_prompt.test_sample(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")

'fallacy task. Topic: Should we allow animal testing? Text: Animal testing abuses animals and should be dis-continued This contains the fallacy: <mask>'

### Load Model
We load a model we have already trained for this task: here are sample predictions

In [118]:
pmodel = RobertaPrompt(model = '/Laidlaw Research Project/models/prompt_combined', device = torch.device('cuda'), prompt = argument_prompt)

In [134]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Animal testing abuses animals and should be dis-continued"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: NoFallacy
Stance: contradict


In [142]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Your stupid for bringing this up, animal testing is horrible"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: AdHominem
Stance: contradict


In [144]:
fallacy = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "My Dad had a dog once, he says animal testing should be allowed"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: IrrelevantAuthority
Stance: support


In [148]:
fallacy = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "fallacy")
stance = pmodel.infer(["Should we allow animal testing?", "Everyone has a favorite animal, what is yours?"], "stance")
print("Fallacy: {}\nStance: {}".format(fallacy, stance))

Fallacy: RedHerring
Stance: support


## Train Model

In [198]:
pmodel = RobertaPrompt(model='roberta-large', device = torch.device('cuda'), prompt = argument_prompt)

In [ ]:
pmodel.train("sample_train_set.tsv", "sample_val_set.tsv", output_dir="sample_model", epochs=10)

In [ ]:
pmodel.test("sample_test_set.tsv")